In [1]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [2]:
def setDocumentContent():
    with open('/Users/sameernawaz/AnacondaProjects/Jupyter Notebook/ML Personalisation/Datasets/Streams.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        count = 0
        for line in csv_reader:
            val[count] = line
            count = count + 1
            if count >= sizeOfSets :
                break
            
    countTemp = 0
    for i in val:
        document[countTemp] = i[13]
        countTemp += 1

In [3]:
sizeOfSets = 300
val = [None] * sizeOfSets
document = [""] * sizeOfSets

setDocumentContent()

In [4]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
df = pd.DataFrame(val, columns = ['objectID', 'Shares', 'VideoType', 'VideoName', 'videoURL', 
                                  'Comments', 'profileClicks', 'userPointerType', 'userPointerClassName', 
                                  'userPointerObjectID', 'imageType', 'imageName', 'imageURL', 'text', 
                                  'likes', 'views', 'bookmarks', 'createdAt', 'updatedAt']) 
df.head()

,objectID,Shares,VideoType,VideoName,videoURL,Comments,profileClicks,userPointerType,userPointerClassName,userPointerObjectID,imageType,imageName,imageURL,text,likes,views,bookmarks,createdAt,updatedAt
0,gFfpVo2FwT,0,,,,1,0,Pointer,_User,JdOjsewt0L,,,,"Cheers ladies! Pleasure to meet you, I’m Safin...",7,12,0,2020-03-03T12:49:31.508Z,2020-03-03T12:49:31.508Z
1,jRNVujUa03,0,,,,0,1,Pointer,_User,XJ1fbtm3aC,File,652013e6864a6059b449f6b081709304_image.jpg,https://parsefiles.back4app.com/Eu2H7JiGkiBwPa...,Transform your life! Tomorrow Saturday Novembe...,6,7,1,2020-03-03T12:49:31.508Z,2020-03-03T12:49:31.508Z
2,uvvYbZtcsl,0,,,,1,0,Pointer,_User,MPIYycMk29,File,945e66a01ae64098f2b94937a134a566_image.jpg,https://parsefiles.back4app.com/Eu2H7JiGkiBwPa...,Making moves at the Digital Hollywood conventi...,6,22,0,2020-03-03T12:49:31.517Z,2020-03-03T12:49:31.517Z
3,h7GgZoqjRL,0,,,,0,1,Pointer,_User,EwlsOjH8zR,File,791db236ef99b9603c5dd2e6b5265292_image.jpg,https://parsefiles.back4app.com/Eu2H7JiGkiBwPa...,Note to self: All your milestones started from...,7,16,0,2020-03-03T12:49:31.517Z,2020-03-03T12:49:31.517Z
4,uFsYWSv0uZ,0,,,,1,1,Pointer,_User,DkROZ0bjmt,,,,Hello! So excited to be part of this! I’m Alis...,6,9,0,2020-03-03T12:49:31.517Z,2020-03-03T12:49:31.517Z


In [6]:
df = df[['objectID', 'userPointerObjectID', 'text']]
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 3 columns):
objectID               300 non-null object
userPointerObjectID    300 non-null object
text                   300 non-null object
dtypes: object(3)
memory usage: 9.4+ KB


In [7]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['text']))
len(norm_corpus)

300

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(300, 1953)

In [131]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.000000,0.031253,0.000000,0.031348,0.000000,0.000000,0.000000,0.0,0.018056,0.00000,...,0.000000,0.074462,0.0,0.044366,0.0,0.000000,0.000000,0.000000,0.009750,0.038225
1,0.031253,1.000000,0.000000,0.000000,0.000000,0.066673,0.036348,0.0,0.036857,0.02975,...,0.000000,0.000000,0.0,0.000000,0.0,0.067165,0.000000,0.093421,0.019893,0.033656
2,0.000000,0.000000,1.000000,0.000000,0.088863,0.000000,0.000000,0.0,0.000000,0.00000,...,0.000000,0.000000,0.0,0.000000,0.0,0.080018,0.000000,0.085184,0.026671,0.039878
3,0.031348,0.000000,0.000000,1.000000,0.019705,0.000000,0.000000,0.0,0.010130,0.00000,...,0.000000,0.063811,0.0,0.000000,0.0,0.000000,0.000000,0.019158,0.000000,0.098137
4,0.000000,0.000000,0.088863,0.019705,1.000000,0.020010,0.047376,0.0,0.029298,0.03607,...,0.091042,0.000000,0.0,0.000000,0.0,0.017008,0.018261,0.015002,0.037885,0.017022


In [132]:
movies_list = df['objectID'].values
movies_list, movies_list.shape

(array(['gFfpVo2FwT', 'jRNVujUa03', 'uvvYbZtcsl', 'h7GgZoqjRL',
        'uFsYWSv0uZ', 'F4gcyaeB0d', '4nBTSiQwHL', 'FPTz6YhQMb',
        'NvSHNuCrQh', 'EBCE9AXTQD', 's8Ckilgwk2', 'JlBd0kqVou',
        'ssuCcmPnTY', 'lJFFQiUN5l', 'Rf2iiUPE1H', 'LdIk9WxcGC',
        'HfBD0OFTBl', 'KIM2uTbCOz', '79sAJnCNfw', 'QtKac7HzJR',
        'zWzbMtOnfc', 's2p3C7g7SE', 'O1weceVmRs', '8BZVQFVwaT',
        'Ve6EkVJUG8', 'EhDhX22EvM', 'o0ZMcH9HaD', 'EkCSbT3w2x',
        'eJCsyOHrKM', 'VKKCXWAvkp', '7LwwopHrzt', 'NgslXBqwnK',
        'eSXB7yUFzm', 'Lg8DpLrxZL', 'EutKhxv4oF', 'edk4FU0i60',
        'ApBn2koyDG', 'xZ3TVkD04n', 'pAPBBch4rd', 'bQtJMlHovr',
        'MXZNm8JOxi', 'tgzfL8z0lr', 'Q1ReuDcEFT', 'QuIBtMT7fU',
        'TuZmkTGwQL', 'a0hpG87wrb', 'ftgGkuv6Nm', 'jyCdQiz9mA',
        'lAOjGGbAq7', 'bZxWPVTxTZ', 'olrNbG3dGR', 'x6MzHiCdFc',
        'eRgsUaGfF2', '9MDZgDmEiA', 'kltNeVnoCb', 'hOPYqKF7dA',
        'D2KcgpGjXf', 'EiDj2fqnqe', 'EoLb6GHQfp', 'jn3iLya8Kv',
        'CFjjpnRr4t', 'yxc6f6sih3', 'sKb

In [133]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([0.00993323, 0.        , 0.05136864, 0.01827353, 0.04378239,
       0.0287526 , 0.02717086, 0.02605915, 0.03269251, 0.09388537,
       0.00840585, 0.02615348, 0.05336123, 0.01659981, 0.00999785,
       0.03597081, 0.01053826, 0.01065811, 0.01877665, 0.0533933 ,
       0.        , 0.02636924, 0.02714609, 0.01210838, 0.01024766,
       0.10131269, 0.0165404 , 0.00929257, 0.        , 0.04921451,
       0.0551109 , 0.00560375, 0.04495837, 0.07019755, 0.        ,
       0.0908002 , 0.        , 0.        , 0.0870333 , 0.        ,
       0.        , 0.0169487 , 0.01196385, 0.02578376, 0.        ,
       0.        , 0.05022102, 0.03660083, 0.03880366, 0.01532058,
       0.01605611, 0.        , 0.01224879, 0.05758735, 0.        ,
       0.        , 0.04307206, 0.08356037, 0.03891085, 0.0554973 ,
       0.24650806, 0.04669876, 0.01971421, 0.01386701, 0.01593791,
       0.00733273, 0.03032483, 0.01700432, 0.02645692, 0.03050253,
       0.04767125, 0.0763688 , 0.0438506 , 0.        , 0.     

In [134]:
movie_idx = np.where(movies_list == 'N2itQQuFDN')[0][0]
movie_idx

136

In [135]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([103,  60,  94, 209, 129])

In [136]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array(['tUBOthLDa9', 'CFjjpnRr4t', 'wAXg1N4OZn', 'aa9JsX1Fp8',
       'yFuDaTvAKb'], dtype=object)

In [137]:
# print((df.loc[df['objectID'] == "nDP1lNU6xF"]['text']).values[0])

In [138]:
# for i in range(5):
#     print("=>",(df.loc[df['objectID'] == similar_movies[i]]['text']).values[0])

In [139]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

In [148]:
popular_movies = ['eJKaktxEDp', 'jCNLOfJwUu', 'GlDtJJVe7T', '5Ux17Uqlhp', 'CrbJfY0bGA']

In [149]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))

Movie: eJKaktxEDp
Top 5 recommended Movies: ['LdIk9WxcGC' 'jaT0IX5lCj' 'Oqmo29bysT' 'hRNdsgmnrf' 'M3wn4zVxqF']
Movie: jCNLOfJwUu
Top 5 recommended Movies: ['yFuDaTvAKb' 'UjTGMDLe9N' 'wAXg1N4OZn' 'BI8NvznRwM' 'ZgJMEtLtSz']
Movie: GlDtJJVe7T
Top 5 recommended Movies: ['0AqusRrLK2' 'eRgsUaGfF2' 'JglKpEXObE' 'pAPBBch4rd' 'POUqTCTIV2']
Movie: 5Ux17Uqlhp
Top 5 recommended Movies: ['EKMKT1lNRS' 'aFNX7Bhk9u' 'vwUSAw23ng' 'Gi3pPJIbxJ' 'rfueXDEpRw']
Movie: CrbJfY0bGA
Top 5 recommended Movies: ['PVvcGjXp2S' '0zNBvO3QGr' 'SKu3atdfh7' '2tI4crzwET' 'cDGXAunEKK']


In [147]:
print((df.loc[df['objectID'] == "nDP1lNU6xF"]['text']).values[0])
print('\n')
for i in movie_recommender(movie_title = "nDP1lNU6xF", movies=movies_list, doc_sims=doc_sim_df):
    print("=>",(df.loc[df['objectID'] == i]['text']).values[0])

Happy Holidays Beauties!!! For those of you celebrating Christmas Eve today I hope you’re all fully immersed in piles of cookies and eggnog up to your eyeballs! For those of you celebrating Hanukkah, Kwanzaa, or other beautiful traditions, I hope you are surrounded by meaningful new memories and precious family and friends! Enjoy this time and remember to BE IN THE PRESENT 🎁 (pun intended 🤓)


=> Hey ladies! Just wanted to quickly pop in to wish everyone a great Monday and rest of the week! ✨ Whatever your goal is today I hope you crush it! 💥
=> Holidays are bittersweet for most. May you enjoy the moment, be thankful for memories and never forget there is work to be done to reach those goals! XoXo #MarketingRealtor 
=> With work, our business or side hustles, and social media being the center of everything, it’s hard to unplug and just be with the ones you love this Christmas season. I’m equally as guilty. But this year, I’m making a conscious effort to be intentional with my time\n\nI